In [ ]:
## import packages
import pystac_client
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import geopandas as gpd
import os

In [ ]:
## load pp statistics data
catalog = pystac_client.Client.open(
    "https://storage.googleapis.com/coclico-data-public/coclico/coclico-stac/catalog.json"
)

collection = catalog.get_child(id = 'pp_stats')
cloud_lau_path = collection.assets['geoparquet-stac-items'].href

df = gpd.read_parquet(cloud_lau_path)

In [ ]:
## choose parameters (this is what you would do in drop down of the platform)
lau = 'DE_03352011' # GISCO_ID of Cuxhaven, Stadt
dmap = 'UNDEFENDED_MAPS' # choose level of defence
rp = '1000' # choose return period
scenario = 'SSP585' # choose integrated scenario
y = 2030 # choose the year

In [ ]:
## Filter and rearrange dataframe for easier plotting
# filter data for one LAU based on the unique GISCO_ID
df_lau = df[df['GISCO_ID']==lau] 

# Extract column parameters
columns_split = [col.split("\\") for col in df_lau.columns]
columns_df = pd.DataFrame(columns_split, columns=["map", "rp", "slr", "year", "pop","statistic"])

# Add extracted columns to the original DataFrame for easier filtering
df_tidy = df_lau.T.reset_index(drop=True)  # Transpose data for easier use
df_tidy = df_tidy.rename(columns={df_tidy.columns[0]: "exposed_population"})
df_tidy = pd.concat([columns_df, df_tidy], axis=1)

# filter data for the parameter choices made earlier
df_subset = df_tidy[df_tidy['map']==dmap]
df_subset = df_subset[df_subset['rp']==rp]
df_subset = df_subset[df_subset['year']!= '2150'] # neglect year 2150
# filter for the relative exposed population, as that is what we want to visualize
df_subset = df_subset[df_subset['statistic'] == 'rel_affected'] 

# Ensure year and exposed_population are numeric
df_subset["year"] = pd.to_numeric(df_subset["year"], errors="coerce")
df_subset["exposed_population"] = pd.to_numeric(df_subset["exposed_population"], errors="coerce")

# Filter out rows with NaN in exposed_population
df_subset = df_subset.dropna(subset=["exposed_population"])

In [ ]:
## Create line plot for the selected flood maps
# Assigning colors for each integrated scenario
color_dict = {
    'SSP126': '#2F8FCE',
    'SSP245': '#0B4572',
    'None': '#EFC530',
    'High_End': '#DF9239',
    'SSP585': '#C7432B'
}

# Make a line plot of exposed population development over time for each integrated scenario
plt.figure(figsize=(12, 6))
sns.lineplot(
    data=df_subset, 
    x="year", 
    y="exposed_population", 
    hue="slr", 
    marker="o",
    palette=color_dict
)

# Highlight the specific data point, that has been selected in the drop-down and is now visible on the map
highlight_point = df_subset[(df_subset["year"] == y) & (df_subset["slr"] == scenario)]
if not highlight_point.empty:
    plt.scatter(
        highlight_point["year"], 
        highlight_point["exposed_population"], 
        color="black", 
        s=100,  # Size of the marker
        marker="x", 
        linewidth=2,
        zorder=5,
        label="Visible in the map"
    )

# Customize plot
plt.title(f"Exposed Population in {df_lau['LAU_NAME'].item()}")
plt.xlabel("Year")
plt.ylabel("Relative Exposed Population")
plt.legend(title="SLR Scenario", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid()
plt.tight_layout()
years = [2010,2030,2050,2100]
plt.xticks(ticks=years, labels=[str(year) for year in years])

plt.show()